In [ ]:
import sys

!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install gym

import os
import random
from typing import Dict, List, Tuple
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if not os.path.exists("segment_tree.py"):
    #Download the segment tree module which is needed for PER implementation
    !wget https://raw.githubusercontent.com/curt-park/rainbow-is-all-you-need/master/segment_tree.py
        
from segment_tree import MinSegmentTree, SumSegmentTree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


Prioritized Memory Replay - extension from standard Replay Memory buffer


In [ ]:
#https://github.com/Curt-Park/rainbow-is-all-you-need/blob/master/03.per.ipynb
class PrioritizedReplayBuffer():
    
    def __init__(self, obs_dim, size, batch_size = 32,alpha = 0.6):

        """Initialization."""
        super(PrioritizedReplayBuffer, self).__init__()

        self.obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.next_obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0

        self.max_priority, self.tree_ptr = 1.0, 0
        self.alpha = alpha
        
        # capacity must be positive and a power of 2.
        tree_capacity = 1
        while tree_capacity < self.max_size:
            tree_capacity *= 2

        self.sum_tree = SumSegmentTree(tree_capacity)
        self.min_tree = MinSegmentTree(tree_capacity)

    def __len__(self):
        return self.size

    def store(self, obs, act, rew, next_obs, done):
      
        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

        self.sum_tree[self.tree_ptr] = self.max_priority ** self.alpha
        self.min_tree[self.tree_ptr] = self.max_priority ** self.alpha
        self.tree_ptr = (self.tree_ptr + 1) % self.max_size

    def sample_batch(self, beta: float = 0.4):
        """Sample a batch of experiences."""
        
        indices = self._sample_proportional()
        
        obs = self.obs_buf[indices]
        next_obs = self.next_obs_buf[indices]
        acts = self.acts_buf[indices]
        rews = self.rews_buf[indices]
        done = self.done_buf[indices]
        weights = np.array([self._calculate_weight(i, beta) for i in indices])
        
        return dict(
            obs=obs,
            next_obs=next_obs,
            acts=acts,
            rews=rews,
            done=done,
            weights=weights,
            indices=indices,
        )
        
    def update_priorities(self, indices, priorities):
        """Update priorities of sampled transitions."""


        for idx, priority in zip(indices, priorities):
            self.sum_tree[idx] = priority ** self.alpha
            self.min_tree[idx] = priority ** self.alpha

            self.max_priority = max(self.max_priority, priority)
            
    def _sample_proportional(self):
        """Sample indices based on proportions."""
        indices = []
        p_total = self.sum_tree.sum(0, len(self) - 1)
        segment = p_total / self.batch_size
        
        for i in range(self.batch_size):
            a = segment * i
            b = segment * (i + 1)
            upperbound = random.uniform(a, b)
            idx = self.sum_tree.retrieve(upperbound)
            indices.append(idx)
            
        return indices
    
    def _calculate_weight(self, idx, beta):
        """Calculate the weight of the experience at idx."""
        # get max weight
        p_min = self.min_tree.min() / self.sum_tree.sum()
        max_weight = (p_min * len(self)) ** (-beta)
        
        # calculate weights
        p_sample = self.sum_tree[idx] / self.sum_tree.sum()
        weight = (p_sample * len(self)) ** (-beta)
        weight = weight / max_weight
        
        return weight

Dueling network 

In [ ]:
#https://github.com/higgsfield/RL-Adventure/blob/master/3.dueling%20dqn.ipynb
class DuelingCNN_Network(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super(DuelingCNN_Network, self).__init__()

        # set common feature layer
        self.feature_layer = nn.Sequential(
            nn.Linear(in_dim, 128), 
            nn.ReLU(),
        )
        
        # set advantage layer
        self.advantage_layer = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, out_dim),
        )

        # set value layer
        self.value_layer = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
        )


    def forward(self, x):
        feature = self.feature_layer(x)
        
        value = self.value_layer(feature)
        advantage = self.advantage_layer(feature)

        return value + advantage  - advantage.mean(dim=-1, keepdim=True)
       

Agent

In [ ]:
class DQNAgent:

    def __init__(self, env, memory_size, batch_size, target_update, epsilon_decay, max_epsilon = 1.0, min_epsilon = 0.07, gamma = 0.99,

        # PER parameters
        alpha = 0.2,
        beta = 0.6,
        prior_eps = 1e-6,
    ):
       
        obs_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n
        
        self.env = env
        
        self.batch_size = batch_size
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.target_update = target_update
        self.gamma = gamma
        
        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        
        # PER
        self.beta = beta
        self.prior_eps = prior_eps
        self.memory = PrioritizedReplayBuffer(
            obs_dim, memory_size, batch_size, alpha
        )

        # networks: dqn, dqn_target
        self.dqn = DuelingCNN_Network(obs_dim, action_dim).to(self.device)
        self.dqn_target = DuelingCNN_Network(obs_dim, action_dim).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()
        
        # optimizer
        self.optimizer = optim.Adam(self.dqn.parameters())

        # transition to store in memory
        self.transition = list()

    def select_action(self, state):
        """Select an action from the input state."""
        # epsilon greedy policy
        if self.epsilon > np.random.random():
            selected_action = self.env.action_space.sample()
        else:
            selected_action = self.dqn(
                torch.FloatTensor(state).to(self.device)
            ).argmax()
            selected_action = selected_action.detach().cpu().numpy()

        self.transition = [state, selected_action]
        
        return selected_action

    def step(self, action):
        """Take an action and return the response of the env."""
        next_state, reward, done, _ = self.env.step(action)
        self.transition += [reward, next_state, done]
        self.memory.store(*self.transition)
    
        return next_state, reward, done

    def update_model(self):
        """Update the model by gradient descent."""
        # PER needs beta to calculate weights
        samples = self.memory.sample_batch(self.beta)
        weights = torch.FloatTensor(
            samples["weights"].reshape(-1, 1)
        ).to(self.device)
        indices = samples["indices"]

        # PER: importance sampling before average
        elementwise_loss = self._compute_dqn_loss(samples)
        loss = torch.mean(elementwise_loss * weights)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # PER: update priorities
        loss_for_prior = elementwise_loss.detach().cpu().numpy()
        new_priorities = loss_for_prior + self.prior_eps
        self.memory.update_priorities(indices, new_priorities)

        return loss.item()

    #Main training loop    
    def train(self, num_frames):
        #Printing episode data to console 
        print_every = 1
        
        state = self.env.reset()
        update_cnt = 0
        scores = []
        marking  = []
        score = 0
        episode_count = 0 

        for frame_idx in range(1, num_frames + 1):
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            state = next_state
            score += reward
            
            # PER: increase beta
            fraction = min(frame_idx / num_frames, 1.0)
            self.beta = self.beta + fraction * (1.0 - self.beta)

            #When episode is completed log scores and reset variables
            if done:
                state = self.env.reset()
                scores.append(score)
                marking.append(score)

                if episode_count%100 == 0:
                  print("marking, episode: {}, score: {:.1f}, mean_score: {:.2f}, std_score: {:.2f}".format(
                  episode_count, score, np.array(marking).mean(), np.array(marking).std()))
                  marking = []

                if episode_count%print_every==0 and episode_count!=0:
                  print("episode: {}, score: {:.1f}, epsilon: {:.2f}".format(episode_count, score, self.epsilon))
                
                score = 0
                episode_count +=1

            # if training is ready
            if len(self.memory) >= self.batch_size:
                loss = self.update_model()
                update_cnt += 1
                
                #Decrease epsilon
                self.update_epsilon()

                # if hard update is needed
                if update_cnt % self.target_update == 0:
                    self.dqn_target.load_state_dict(self.dqn.state_dict())
                          
        self.env.close()
    
    #Whether to pursue exploration
    def update_epsilon(self):
        self.epsilon = max(
                      self.min_epsilon, self.epsilon - (
                          self.max_epsilon - self.min_epsilon
                      ) * self.epsilon_decay
                  )
    
    def _compute_dqn_loss(self, samples):       

        state = torch.FloatTensor(samples["obs"]).to(self.device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(self.device)
        action = torch.LongTensor(samples["acts"].reshape(-1, 1)).to(self.device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(self.device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(self.device)

        # Make predictions
        state_q_values = self.dqn(state)
        next_states_q_values = self.dqn_target(next_state)

        #Double DQN - updating the DQN loss target as per https://github.com/Curt-Park/rainbow-is-all-you-need/blob/master/02.double_q.ipynb
        curr_q_value = state_q_values.gather(1, action)
        next_q_value = next_states_q_values.gather(1, self.dqn(next_state).argmax(dim=1, keepdim=True)).detach()
        target = (reward + self.gamma * next_q_value * (1 - done)).to(self.device)

        # calculate element-wise dqn loss
        elementwise_loss = F.smooth_l1_loss(curr_q_value, target, reduction="none")

        return elementwise_loss 
 

Set up environment and random seed


In [ ]:
#Configure environment
env_id = "Gravitar-ram-v0"
env = gym.make(env_id)

video_every = 3 #Frequent videos to collect performance results; however this siginificantly slows the run time
env = gym.wrappers.Monitor(env, "./video", video_callable=lambda episode_id: (episode_id%video_every)==0,force=True)

#Reproducible environment and action spaces, do not change lines 6-11 here (tools > settings > editor > show line numbers)
seed = 742
torch.manual_seed(seed)
env.seed(seed)
random.seed(seed)
np.random.seed(seed)
env.action_space.seed(seed)

[742]

Set parameters and begin training


In [ ]:
#Set parameters
num_frames = 2500000 #An episode averages ~1000 frames; training for approx. 2500 episodes until a Google Colab timeout is encountered
memory_size = 100000
batch_size = 32
target_update = 100
#Epsilon initially decays slowly so as to encourage exploration of alternative game strategies
epsilon_decay = 1/10000

In [ ]:
#Begin training the agent
agent = DQNAgent(env, memory_size, batch_size, target_update, epsilon_decay)
agent.train(num_frames)

RuntimeError: ignored